In [13]:
#Imports
import json
import os
from openai import OpenAI
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from EduFlowCallFunctions import *

client = OpenAI(api_key=os.environ["EDUFLOW_OPENAI_API_KEY"])


In [37]:
#initialization models list
GPT_models = [["gpt-3.5-turbo-1106", "gpt-4-1106-preview"],
            ["gpt-3.5-turbo", "gpt-3.5-turbo-0613", "gpt-4-0613", "gpt-4"]]

model = "gpt-3.5-turbo-1106"

In [18]:
key_idea = """On planet Earth, life exists in hostile and extreme environments, and the organisms that survive there are termed extremophiles."""
key_idea_description = """Extremophiles are organisms that thrive in extreme conditions on Earth, such as high temperatures, acidity, alkalinity, or pressure. These environments include locations like deep-sea hydrothermal vents, acidic hot springs, or subglacial lakes in Antarctica. The adaptability of these organisms expands our understanding of life's limits and opens up possibilities for life in extreme conditions beyond Earth."""
theory = """Extremophiles and Their Environments
Definition: Extremophiles are primarily unicellular microbes, including bacteria and archaea, that live in harsh environments​​.
Habitats: These environments include deep-sea hydrothermal vents with high pressure and temperatures (up to 122 °C), volcanic hot springs, deep underground in mines, and in bodies of water that are highly acidic, alkaline, extremely salty, or even radioactive​​.
The Significance of Extremophiles
Adaptation and Survival: Extremophiles have adapted to survive in conditions that are generally detrimental to most life forms. Their existence expands our understanding of the limits and versatility of life.
Implications for Extraterrestrial Life: The study of extremophiles has significant implications for astrobiology. The discovery of diverse microbial communities in extreme environments like subglacial lakes under the Antarctic ice sheet suggests that life might exist in similar extreme conditions elsewhere in the universe, such as on ice-covered moons like Europa​​.
Europa: A Candidate for Extraterrestrial Life
Characteristics of Europa: Europa, a moon of Jupiter, is a primary candidate for extraterrestrial life exploration. It has an ice-covered surface, many kilometers thick, with a possible sub-surface ocean of salty water. The surface of Europa stretches and relaxes due to tidal movements caused by its elliptical orbit around Jupiter, generating heat that could keep a sub-surface ocean in liquid state​​.
Potential Habitability: Europa is considered a promising location for life beyond Earth due to its thin ice shell, liquid ocean, and contact with a geologically active rocky core. The moon has water and the right chemical elements, along with a potentially stable environment​​.
Broader Understanding
Cellular Requirements for Life: For life to exist, certain conditions are necessary, including the availability of an energy source, liquid water for biochemical reactions, and essential chemical building blocks like carbon, oxygen, nitrogen, and hydrogen. Life can thrive in stable environmental conditions within a specific range of pressure, temperature, light intensity, pH, and salinity​​.
Challenging Traditional Concepts: Extremophiles challenge the traditional concept of habitable environments, indicating that life can exist in conditions previously thought to be uninhabitable. This expands our understanding of possible habitats for life, both on Earth and potentially in extraterrestrial settings.
This comprehensive overview illustrates the remarkable adaptability of life and opens up new possibilities for understanding life's potential existence in extreme environments, both on Earth and beyond."""
student_year_level = "Year 11"
subject = "Biology"
question_difficulty = "moderate"
question_examples_easy = ["What is an extremophile? An extremophile is an organism that thrives in extreme environmental conditions that are typically detrimental to most life forms.", "Give one example of an extreme environment on Earth. One example is deep-sea hydrothermal vents.", "What is unique about Europa, one of Jupiter's moons? Europa has an ice-covered surface and is thought to have a sub-surface ocean of salty water."]
question_examples_moderate = ["How do extremophiles survive in high temperatures? Extremophiles in high temperatures have specialized proteins and enzymes that remain stable and functional at elevated temperatures.", "Why is the study of extremophiles important? Studying extremophiles helps understand the adaptability and limits of life, and it may provide insights into the possibility of life on other planets or moons.", "Why is Europa considered a likely place to find life beyond Earth? Europa is considered likely because of its ice shell, liquid ocean, and geologically active core, which provide conditions potentially suitable for life."]
question_examples_hard = ["Explain how extremophiles challenge the traditional concept of habitable environments. Extremophiles challenge the traditional concept by surviving in environments previously thought to be uninhabitable, expanding our understanding of where life can exist", "Discuss the potential implications of finding life in subglacial lakes in Antarctica for the search for extraterrestrial life. The discovery in subglacial lakes suggests that similar extreme, ice-covered environments in the solar system, such as on Europa, could also harbor life.", "How does the study of extremophiles contribute to our understanding of the potential for life on planets and moons outside the 'Goldilocks Zone'? The study of extremophiles indicates that life can exist in conditions not reliant on a star's warmth, suggesting that planets and moons outside the 'Goldilocks Zone' could also support life."]

student_interests = ["gaming", "politics", "philosophy"]
student_career_goals = ["lawyer", "doctor", "engineer"]

# Functions

In [38]:
def generate_question(key_idea,
                      key_idea_description,
                      theory,
                      student_year_level,
                      subject,
                      question_difficulty,
                      question_examples = [],
                      other_system_instructions = "",
                      student_interests = [],
                      student_career_goals = []):
    """
    Generates a question for a student based on various inputs.

    Parameters:
    key_idea (str): The concept(s) on which the generated question should be about.
    key_idea_description (str): The further description of the key idea.
    theory (str): Additional theory related to the key idea.
    student_year_level (str): The year level of the student.
    subject (str): The subject of the question is based on.
    question_difficulty (str): The difficulty level of the question.
    question_examples (str): Examples questions that are about the same key idea and at a similar difficulty.
    other_system_instructions (str, optional): Other instructions for the system. Defaults to "".
    student_interests (list, optional): A list of the student's interests. Defaults to [].
    student_career_goals (list, optional): A list of the student's career goals. Defaults to [].
    model (str, optional): The model used to generate the question. Defaults to "gpt-3.5-turbo-1106".

    Returns:
    str: generated question"""

    system_prompt = f"""You are an advanced AI agent specialized in generating questions for high school students to answer. Your task is to formulate a question that aligns with the following parameters:

        - The Key Idea: {key_idea}
        - Here is the extended description of the key idea: {key_idea_description}
        - Here is the theory that is taught in the class to this student, from which the question should be based on: {theory}
        - The subject which question falls under: {subject}
        - The year level of the atudent answering the question. Try not to make the question ask content which a what a student at this level would not be able to answer: {student_year_level}
        - Additional Instructions: {other_system_instructions}

    Consider the student's interests and career aspirations in crafting the question, making it engaging and relevant to their future goals. If the interests or career goals do not align directly with the key idea, prioritize the academic integrity of the question.

        - Student Interests: {student_interests}
        - Student Career Goals: {student_career_goals}

    Your response should directly pose the question without prefacing it as a 'question'. Ensure that the question is challenging yet appropriate for the student's year level and subject matter, fostering critical thinking and application of the key idea and related theory.

    You will be prompted to generate a question based on a specific difficulty and will be provided example questions of a similar difficulty. Use these and the guidelines provided above to generate the question."""

    initial_user_prompt = f"""Create a question for the student to answer. Your question should be {question_difficulty}. Here is a list of example questions at a similar difficulty with included answers, which you can losely base your question off: {question_examples}"""

    generated_question = client.chat.completions.create(
        model = "gpt-3.5-turbo-1106",
        messages=[{"role": "system", "content": system_prompt},
                {"role": "user", "content": initial_user_prompt}],
        temperature=0.7)


    return generated_question

In [44]:
generated_question = generate_question(key_idea,
                            key_idea_description,
                            theory,
                            student_year_level,
                            subject,
                            question_difficulty,
                            question_examples = question_examples_moderate,
                            other_system_instructions = "",
                            student_interests = student_interests,
                            student_career_goals = student_career_goals)




In [45]:
generated_question.choices[0].message.content

'How do extremophiles challenge traditional concepts of habitable environments, and what implications does this have for the search for extraterrestrial life, especially in the context of Europa?'

In [46]:
def answer_question(question,
                    key_idea,
                    key_idea_description,
                    student_year_level,
                    theory,
                    other_system_instructions = "",
                    model = "gpt-3.5-turbo-1106"):
    """
    Generates an answer for a given question one would see in a highschool textbook or exam.

    Parameters:
    question (str): The question to be answered.
    key_idea (str): The main concept the question is assessing.
    key_idea_description (str): The description of the key idea.
    answer_year_level (str): The level of the student answering the question.
    subject (str): The subject of the question the question is based on.
    theory (str): The theory related to the question.
    other_system_instructions (str, optional): Other instructions for the system. Defaults to "".
    model (str, optional): The model used to generate the answer. Defaults to "gpt-4".

    Returns:
    str: The generated answer.
    """

    # Put together the things in a decent system instruction
    system_prompt = f"""As an AI designed to assist in educational contexts, your task is to provide an accurate and comprehensive answer to a high school level question. You should base your answer on the following detailed information:

    - Key Idea: {key_idea}. This is the primary concept that the question is exploring.
    - Detailed Description of Key Idea: {key_idea_description}. Use this information to ensure that your answer fully encompasses the nuances of the key idea.
    - Relevant Theory: {theory}. Your answer should incorporate and demonstrate an understanding of this theory, as it relates to the key idea.
    - Student Year Level: {student_year_level}. Tailor the sophistication and depth of your answer to be appropriate for a student at this academic level.
    - Additional Instructions: {other_system_instructions}. Consider these instructions to guide the style, format, or additional content of your answer.

    Your response should directly address the question, formulated in a clear and concise manner, suitable for the specified student year level. Avoid prefacing your response with 'Answer:' or similar qualifiers. Instead, provide a straightforward explanation or solution that demonstrates a comprehensive understanding of the topic."""


    initial_user_prompt = f"Please provide an answer to this question at an appropriate depth, considering the student's academic level: {question}"

    generated_answer = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": initial_user_prompt},
        ],
        temperature = 0.1
    )

    return generated_answer

In [48]:
question = generated_question.choices[0].message.content

generated_answer = answer_question(question,
                    key_idea,
                    key_idea_description,
                    student_year_level,
                    theory,
                    other_system_instructions = "",
                    model = "gpt-3.5-turbo-1106")

In [49]:
generated_answer.choices[0].message.content

"Extremophiles challenge traditional concepts of habitable environments by thriving in conditions that were previously considered uninhabitable for life. These organisms, primarily unicellular microbes like bacteria and archaea, have adapted to survive in extreme environments on Earth, such as high temperatures, acidity, alkalinity, or pressure. This challenges the traditional notion of habitable environments, which typically focused on moderate temperature, pH, and pressure ranges.\n\nThe implications of extremophiles for the search for extraterrestrial life are significant, especially in the context of Europa, a moon of Jupiter. Europa is considered a promising location for extraterrestrial life exploration due to its potential habitability. The moon has an ice-covered surface with a possible sub-surface ocean of salty water. The tidal movements caused by its elliptical orbit around Jupiter generate heat, potentially keeping the sub-surface ocean in a liquid state. This environment, 

In [ ]:
system_prompt = f"""You are an AI developed for educational assessment, tasked with evaluating a student's answer to a question and providing feedback. Your assessment should be based on the following criteria:

    - Key Idea: {key_idea}. This is the central concept the question and answer revolve around.
    - Relevant Theory: {theory}. Evaluate how well the answer reflects an understanding of this theory in relation to the key idea.
    - Detailed Description of Key Idea: {key_idea_description}. Use this to gauge the depth and accuracy of the student's answer.

Format your assessment as a JSON object, which includes a numerical score out of 100, and constructive feedback tailored for the student. Your feedback should guide the student on areas of strength and suggestions for improvement.

    JSON Format:
    {{
        "score": "your_score_here",
        "feedback": "your_feedback_here"
    }}

Avoid explicitly stating "Feedback:". Instead, provide your evaluation and guidance directly as if speaking to the student. Ensure your feedback is clear, supportive, and offers specific insights on the answer.

    Additional System Instructions: {other_system_instructions}"""
